# Alle Karteringen omzetten

Maar voor nu nog even enkel groningen

### Imports etc

In [1]:
from pathlib import Path
import pandas as pd
import geopandas as gpd

import sys
sys.path.append('../')

from veg2hab.waswordtlijst import WasWordtLijst, opschonen_was_wordt_lijst
from veg2hab.definitietabel import DefinitieTabel, opschonen_definitietabel
from veg2hab.vegkartering import Kartering
from veg2hab.fgr import FGR

pd.set_option('display.max_columns', 100)

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


### Omzetclasses instantieren

In [2]:
path_in_wwl = Path("../data/5. Was-wordt-lijst-vegetatietypen-en-habitattypen-09-02-2021.xlsx")
path_out_wwl = Path("../testing/opgeschoonde_waswordt.xlsx")
opschonen_was_wordt_lijst(path_in_wwl, path_out_wwl)
wwl = WasWordtLijst.from_excel(path_out_wwl)

path_in_dt = Path("../data/definitietabel habitattypen (versie 24 maart 2009)_0.xls")
path_in_mitsjson = Path("../data/mitsjson.csv")
path_out_dt = Path("../testing/opgeschoonde_definitietabel.xlsx")
opschonen_definitietabel(path_in_dt, path_in_mitsjson, path_out_dt)
dt = DefinitieTabel.from_excel(path_out_dt)

fgr = FGR(Path("../data/bronbestanden/FGR.json"))

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Alle SBB codes zijn valide
Alle VvN codes zijn valide
Alle SBB codes zijn valide
Alle VvN codes zijn valide
Alle SBB codes zijn valide
Alle VvN codes zijn valide


### Inladen benodigde informatie

**Overzicht_vegetatiekarteringen.xlsx** is uit de drive en weet van karteringen de provincie, naam, shapefile pad en of het een access of een shapefile kartering is.

**accesskarteringinfo.xlsx** bevat voor de access karteringen de informatie die nodig is om een access kartering in te laden.

**shapefilekarteringinfo.xlsx** bevat voor de shapefile karteringen de informatie die nodig is om een shapefile kartering in te laden.


In [3]:
# info over de karteringen
overzicht = pd.read_excel(
    "../data/Overzicht_vegetatiekarteringen.xlsx", 
    sheet_name="Vegetatiekarteringen", 
    engine='openpyxl',
    usecols=[
        "provincie", 
        "naam_kartering", 
        "path_shapes", 
        "landelijke_typologie beschikbaar?", 
        "locatie_landelijke_typologie",
    ],
    dtype="string"
)

# benodigde info voor de access karteringen
access_info = pd.read_excel(
    "../data/accesskarteringinfo.xlsx",
    engine='openpyxl',
    usecols=[
        "naam_kartering",
        "path_csvs",
        "elm_id_name",
    ],
    dtype="string"
)

# benodigde info voor de shapefile karteringen
shape_info = pd.read_excel(
    "../data/shapefilekarteringinfo.xlsx",
    engine='openpyxl',
    usecols=[
        "naam_kartering",
        "ElmID_col",
        "VvN_col",
        "SBB_col",
        "vegtype_split_char",
        "datum_col",
        "opmerking_col",
    ],
    dtype="string"
)
# Convert all pd.NA to None to prevent having to do "if not pd.isna()" everywhere
shape_info = shape_info.applymap(lambda x: None if pd.isna(x) else x)

# Voor nu dus even enkel karteringen uit groningen
overzicht = overzicht[overzicht.provincie == "Groningen"]
overzicht = overzicht[overzicht["landelijke_typologie beschikbaar?"] == "Ja"]
overzicht.path_shapes = overzicht.path_shapes.apply(lambda x: "../testing/vegetatiekarteringen" + x[1:])

access_overzicht = overzicht[overzicht["locatie_landelijke_typologie"] == "access"]
access_overzicht = access_overzicht.join(access_info.set_index("naam_kartering"), on="naam_kartering")
access_overzicht.path_csvs = access_overzicht.path_csvs.apply(lambda x: "../testing/vegetatiekarteringen" + x[1:])

shapefile_overzicht = overzicht[overzicht["locatie_landelijke_typologie"] == "in shapes"]
shapefile_overzicht = shapefile_overzicht.join(shape_info.set_index("naam_kartering"), on="naam_kartering")

### Beddekking/Percentages in shapefilekarteringen...

...doen we voor nu nog even niets mee.

Ze zijn of altijd 100 of ook boven de 100, en ze zijn toch niet per vegtypeinfo beschikbaar maar per vlak

In [4]:
for row in shapefile_overzicht.itertuples():
    t = gpd.read_file(row.path_shapes)
    print(row.naam_kartering)
    print(t.BEDEKKING.value_counts())

SBB ZWK 2010 0814_Tussen de Gasten 2010
100.0    2007
Name: BEDEKKING, dtype: int64
SGL Hunzedal en Leekstermeer2021 Hunzedal
100.0    1773
103.0      22
109.0       2
106.0       2
90.0        1
Name: BEDEKKING, dtype: int64
SGL Hunzedal en Leekstermeer2021 Leekstermeer 2021
100.0    271
Name: BEDEKKING, dtype: int64


### Karteringen inladen

In [5]:
karteringen = {}
for row in access_overzicht.itertuples():
    karteringen[row.naam_kartering] = Kartering.from_access_db(Path(row.path_shapes), row.elm_id_name, Path(row.path_csvs))

for row in shapefile_overzicht.itertuples():
    karteringen[row.naam_kartering] = Kartering.from_shapefile(
        Path(row.path_shapes),
        row.ElmID_col,
        VvN_col=row.VvN_col,
        SBB_col=row.SBB_col, 
        vegtype_split_char=row.vegtype_split_char, 
        datum_col=row.datum_col, 
        opmerking_col=row.opmerking_col
    )

Er zijn 7 vlakken zonder VegTypeInfo. Deze worden verwijderd.
De eerste paar ElmID van de verwijderde vlakken zijn: [6099, 8174, 8174, 11051, 11023]
Er zijn 90 vlakken zonder VegTypeInfo. Deze worden verwijderd.
De eerste paar ElmID van de verwijderde vlakken zijn: [78, 76, 69, 82, 81]


### Karteringen omzetten

In [6]:
for kartering in karteringen.values():
    kartering.apply_wwl(wwl)

In [7]:
for kartering in karteringen.values():
    kartering.apply_deftabel(dt)

In [8]:
for kartering in karteringen.values():
    kartering.check_mitsen(fgr)

### Karteringen wegschrijven

In [9]:
for gebied in (karteringen.keys()):
    print(gebied)
    p = Path(f"../testing/omgezette_vegetatiekarteringen/GR/{gebied}.shp")
    karteringen[gebied].final_format_to_file(p)

NM vegetatiekartering RuitenAa2020
SBB Duurswold2013
SBB Lauwersmeer 2015
SBB Westerwolde 2020
SBB ZWK 2010 0815_Trimunt_2010
SGL Zuidlaardermeer 2019
SBB ZWK 2010 0814_Tussen de Gasten 2010
SGL Hunzedal en Leekstermeer2021 Hunzedal
SGL Hunzedal en Leekstermeer2021 Leekstermeer 2021
